In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
acc_features_for_all_attributes.npy
berk.csv
berk-test.csv
GoogleLenetv2.png
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv
weighted_acc_features_for_all_attributes.npy


# Preparing attributes_list & labels

In [2]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_RAP/RAP_annotation/RAP_annotation.mat')['RAP_annotation']
attributes_list = []
for i in range(data['attribute_eng'][0][0].shape[0]):
    attributes_list.append(data['attribute_eng'][0][0][i][0][0])
print(attributes_list)
attributes_list_chinese = []
for i in range(data['attribute_chinese'][0][0].shape[0]):
    attributes_list_chinese.append(data['attribute_chinese'][0][0][i][0][0])
print(attributes_list_chinese)

['Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand', 'faceFront', 'faceBack', 'faceLeft', 'faceRight', 'occlusionLeft', 'occlusionRight', 'occlusionUp', 'occlusionDown', 'occlusion-Environment', 'occlusion-Attachment', 'occlusion-Person', 'occlusion-Other', 'up-Black

In [3]:
low_level = [11]
mid_level = [9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
high_level = [0,1,2,3,4,5,6,7,8,43,44,45,46,47,48,49,50]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  51
['hs-BlackHair', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand']


In [4]:
filename = r"../results/RAP_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 51))
for i in range(length):
    data_y[i] = np.array(data[i, 1:52], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
split = np.load('../results/RAP_partion.npy').item()
y_test = data_y[list(split['test'][0])]
y_test_hiar = data_y_hiar[list(split['test'][0])]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)
low_level_new = [11]#,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91
mid_level_new = [4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
high_level_new = [0,1,2,3]#,51,52,53,54,55,56,57,58,59,60,61,62
data_y_hiar_new = data_y[:, list(np.hstack((low_level_new, mid_level_new, high_level_new)))]
y_test_hiar_new = data_y_hiar_new[list(split['test'][0])]

The shape of the y_test is:  (8317, 51)
The shape of the y_test is:  (8317, 51)


In [5]:
print(split)

{'train': [array([ 5210, 17912, 25728, ...,  1981, 22172,  3646], dtype=uint16), array([ 6961, 35738, 25784, ..., 18723, 40502, 22097], dtype=uint16), array([13242, 39282, 19537, ..., 38389,  9297,  5506], dtype=uint16), array([35992,  8580, 30346, ..., 28795,  2357, 26498], dtype=uint16), array([36907, 15156,  8192, ..., 20005, 16674, 21390], dtype=uint16)], 'test': [array([22911,   707, 15027, ..., 20882, 25363, 40023], dtype=uint16), array([24594, 29503,  1765, ..., 11219, 21624,  9617], dtype=uint16), array([30342, 35736, 20362, ..., 38957,  4655, 36643], dtype=uint16), array([ 4652, 23823, 14049, ..., 24581, 14835,  6691], dtype=uint16), array([20114,   965, 31554, ..., 36422, 11673,  9655], dtype=uint16)]}


In [6]:
"""
print(attributes_list_hiar[6:15])
for i in range(8317):
    if sum(y_test_hiar[i, 6:15]) != 1:
        print(y_test_hiar[i, 6:15])
"""
"""
print(attributes_list_hiar[15:21])
for i in range(8317):
    if sum(y_test_hiar[i, 15:21]) != 1:
        print(y_test_hiar[i, 15:21])
"""
"""
print(attributes_list_hiar[21:26])
for i in range(8317):
    if sum(y_test_hiar[i, 21:26]) != 1:
        print(y_test_hiar[i, 21:26])
"""
"""
si = 26
ei = 34
print(attributes_list_hiar[si:ei])
for i in range(8317):
    if sum(y_test_hiar[i, si:ei]) != 1:
        print(y_test_hiar[i, si:ei])
"""
#"""
si = 35
ei = 38
print(attributes_list_hiar[si:ei])
for i in range(8317):
    if sum(y_test_hiar[i, si:ei]) != 1:
        print(y_test_hiar[i, si:ei])
#"""
"""
si = 38
ei = 41
print(attributes_list_hiar[si:ei])
for i in range(8317):
    if sum(y_test_hiar[i, si:ei]) != 1:
        print(y_test_hiar[i, si:ei])
"""
"""
si = 41
ei = 43
print(attributes_list_hiar[si:ei])
for i in range(8317):
    if sum(y_test_hiar[i, si:ei]) != 1:
        print(y_test_hiar[i, si:ei])
"""
"""
si = 43
ei = 51
print(attributes_list_hiar[si:ei])
for i in range(8317):
    if sum(y_test_hiar[i, si:ei]) != 1:
        print(y_test_hiar[i, si:ei])
"""

['AgeLess16', 'Age17-30', 'Age31-45']


'\nsi = 43\nei = 51\nprint(attributes_list_hiar[si:ei])\nfor i in range(8317):\n    if sum(y_test_hiar[i, si:ei]) != 1:\n        print(y_test_hiar[i, si:ei])\n'

# Positive Samples

In [7]:
for i in range(data_y_hiar.shape[1]):
    print(sum(data_y_hiar[:, i]))

39288.0
158.0
7873.0
695.0
3034.0
435.0
9198.0
5133.0
2403.0
9517.0
7185.0
13107.0
921.0
1260.0
742.0
23302.0
2708.0
1164.0
1523.0
11159.0
5544.0
12578.0
11265.0
5159.0
517.0
6543.0
754.0
2669.0
1169.0
1753.0
1146.0
526.0
1019.0
12251.0
12951.0
415.0
17882.0
23288.0
6027.0
30876.0
4064.0
39352.0
2008.0
1418.0
1364.0
3818.0
1021.0
432.0
732.0
975.0
5361.0


In [8]:
print(attributes_list[11])
print(attributes_list[24])
print(attributes_list[2])

hs-BlackHair
lb-LongTrousers
Age17-30


# Ratio>0.01

In [9]:
selected_ = []
selected = []

total = len(data_y)
print(total)
for i in range(len(attributes_list_hiar)):
    if i!=1:
        selected_.append(i)
    else:
        print(attributes_list[i])
for i in range(len(attributes_list_hiar)):
    if i!=len(low_level)+len(mid_level)+1:
        selected.append(i)
    else:
        print(attributes_list_hiar[i])
print(len(selected_))
print(len(selected))

41585
AgeLess16
AgeLess16
50
50


# Defining the function

In [10]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0.).count(True)
        #print(TP, P, TN, N)
        #print(P,',', N,',', TP,',', TN)
        #if P != 0:
        res += TP/P + TN/N
        """else:
            res += 1 + TN/N"""
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            #print(y_pred[i])
            #print(y_true[i])
            #print(sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i])))
            M_ += 1
    #print(M_)
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Metrics

In [22]:
selected = [0,30,31,32,33,34,35,36,3,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79]
selected = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,2,28,29,30,31,32,33,34,35,36,37,41,42]
###hiarBayesGoogLeNet
#newlossnoexp_epoch50
predictions_prob = np.load(
        "../results/predictions/hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_epoch50_valloss0.43_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
y_test_hiar_new = data_y_hiar_new[list(split['test'][1])]
label = y_test_hiar_new#[:, selected]
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

#"""
###hiarGoogLeNet
#newlossnoexp_epoch75
predictions_prob = np.load(
    "../results/predictions/hiarGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_epoch75_valloss0.35_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar_new#[:, selected]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))
#"""

selected_ = []
###GoogLeNet
predictions_prob = np.load(
    "../results/predictions/GoogLeNet_binary51_75v2_binary51_b4_75_allin_final_0partion_500iter_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = data_y[list(split['test'][1])]#[:, selected_]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.7804300627859513
0.4041977005630466
0.4486888807001381
0.783226061642246
0.5705342355296569
hiarGoogLeNet
0.7804305376260459
0.47010552789401233
0.5335038720523915
0.7784527554125855
0.6331117211167933
GoogLeNet
0.6911396670355735
0.5882254873728049
0.7477219360936318
0.7093688859935613
0.728040653059509


In [17]:
import re
import os

string = [s for s in os.listdir("../results/predictions/") 
          if re.match("hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_*", s)]
string

['hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_final200iter_model_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch100_valloss0.39_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch200_valloss0.31_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch75_valloss0.42_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch25_valloss0.54_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch175_valloss0.32_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch125_valloss0.37_predictions_imagenet_test_RAP.npy',
 'hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split1_epoch50_vallo

In [70]:
###hiarBayesGoogLeNet
#split0_epoch50
#split1_epoch150
#split2_epoch125
#split3_epoch175
#split4_epoch100
for i in range(5):
    files = [s for s in os.listdir("../results/predictions/") 
          if re.match("hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split"+str(i)+"_*", s)]#
    for file in files:
        predictions_prob = np.load("../results/predictions/" + file)
        predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
        y_test_hiar_new = data_y_hiar_new[list(split['test'][i])]
        label = y_test_hiar_new#[:, selected]
        #label = y_test
        print("Split", str(i), file[:-32])
        print(mA(predictions, label), end="\t")
        print(acc(predictions, label), end="\t")
        prec_value = prec(predictions, label)
        print(prec_value, end="\t")
        rec_value = rec(predictions, label)
        print(rec_value, end="\t")
        print(2*prec_value*rec_value/(prec_value+rec_value))

Split 0 hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split0_epoch100_valloss0.39_p
0.7882159564585678	0.4502816274183534	0.5055281562680992	0.7858456751056142	0.6152627621775539
Split 0 hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split0_epoch75_valloss0.41_p
0.7849022039442487	0.4561995683155265	0.505756930626626	0.8065156487935029	0.6216709629281264
Split 0 hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split0_epoch25_valloss0.52_p
0.7438159564059504	0.36981128198797725	0.4148444179276313	0.7509289174147209	0.5344412334812113
Split 0 hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split0_final200iter_model_p
0.503285772029347	0.26999971108228954	0.3501758711322138	0.5317657245230271	0.42227632020180994
Split 0 hiarBayesGoogLeNet_binary51_newhierbinary51_newhier_newlossnoexp_split0_epoch150_valloss0.34_p
0.783332501607738	0.48813951992670923	0.5477865475643577	0.7996113838334459	0.650166293321943
Split 0 hiarBa

# Attributes

In [137]:
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load(
    "../results/predictions/hiarGoogLeNet_binary51_75v2_binary51_75_epoch100_valloss0.11_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, :]#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

"""
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})
"""

###GoogLeNet
predictions_prob = np.load(
    "../results/predictions/GoogLeNetv2_binary51_75v2_binary51_b4_75v2__final_0partion_500iter_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,GoogLeNet
attributes,,
hs-BlackHair,0.943008,0.948299
hs-BaldHead,0.996393,0.996633
hs-LongHair,0.917999,0.911507
hs-Hat,0.980041,0.979800
hs-Glasses,0.912589,0.915595
hs-Muffler,0.988698,0.990381
ub-Shirt,0.879764,0.877720
ub-Sweater,0.866298,0.869424
ub-Vest,0.969821,0.970783


In [138]:
print(np.mean(data_dict['GoogLeNet']))
print(np.mean(data_dict['hiarBayesGoogLeNet']))

0.9099670648588881
0.9086656906359996


In [14]:
df.to_csv("../results/metrics/binary51_imagenet_acc7600_RAP.csv")

In [139]:
def mA_attribute(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true)
    N = M - P
    TP = sum(y_pred*y_true)
    TN = list(y_pred+y_true == 0).count(True)
    #if P != 0:
    res += TP/P + TN/N
    #else:
        #res += TN/N
    return res / 2
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load(
    "../results/predictions/hiarGoogLeNet_binary51_75v2_binary51_75_epoch100_valloss0.11_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

"""
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})
"""

###GoogLeNet
predictions_prob = np.load(
    "../results/predictions/GoogLeNetv2_binary51_75v2_binary51_b4_75v2__final_0partion_500iter_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,GoogLeNet
attributes,,
hs-BlackHair,0.643237,0.656993
hs-BaldHead,0.624819,0.694082
hs-LongHair,0.857269,0.832283
hs-Hat,0.579669,0.576596
hs-Glasses,0.604837,0.621661
hs-Muffler,0.555203,0.527723
ub-Shirt,0.797260,0.806949
ub-Sweater,0.613903,0.613543
ub-Vest,0.802779,0.815007


In [16]:
df.to_csv("../results/metrics/binary51_imagenet_ma7600_RAP.csv")

# In HiarBayesGoogLeNet, Select one each level

In [21]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary3_binary3_epoch50_valloss0.40.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test[:, [11,24,3]]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[1. 1. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 ...
 [1. 1. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 ...
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
0.9478177227365636
0.8421305759288205
0.3824696404953709
0.6308526698426072
0.8337924336926777
0.39862568115648445


In [22]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.6210902615639231
0.6732812186811066
0.8233486017357763
0.8115317720341617
0.817397481171928


# In GoogLeNet, Select one each level

In [29]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_mysplit_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
label = y_test[:, selected_]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]]
0.6878682217145605
0.5467115546471083
0.4471564265961284
0.856679091018396
0.5901166285920404
0.8986413370205603
0.945653480822412
0.9503426716364074
0.9956715161716966
0.8077431766261873
0.9183599855717206
0.9800408801250451
0.9305037874233497
0.9915835036671877
0.7827341589515451
0.8788024528075027
0.9377179271371889
0.7699891787904293
0.8291451244439101
0.35241072502104126
0.9758326319586389
0.9693399062161837
0.9822051220391969
0.5016231814356138
0.9333894433088854
0.9734279187206926
0.9604424672357821
0.7321149452927739
0.8660574726463869
0.7010941445232656
0.7271852831549838
0.8743537333173019
0.9877359624864734
0.8470602380666106
0.9823253577010942
0.939280990741854
0.9722

In [30]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.49995229334997476
0.32598183494673033
0.7176405755280579
0.36504557924541114
0.48392882541009785
